### Analysis

load Employee Salary cvs file

and find the employees with highest salaries, department highest salaries, etc



The csv file has the next columns

* EMPLOYEE_ID	
* FIRST_NAME	
* LAST_NAME	
* EMAIL	
* PHONE_NUMBER	
* HIRE_DATE	
* JOB_ID	
* SALARY	
* COMMISSION_PCT	
* MANAGER_ID	
* DEPARTMENT_ID





In [1]:
from IPython.display        import display, HTML

import findspark
findspark.init()
import pyspark

#from pyspark.context import SparkContext
from pyspark.sql            import SparkSession
from pyspark.sql            import functions as F
from pyspark.sql.functions  import col, explode, udf

from pyspark.sql.window import Window


from pyspark.sql.types      import Row, StringType, LongType, ArrayType



In [2]:
spark = SparkSession.builder \
        .appName( 'employees 50' )  \
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/10 12:13:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### extract

In [3]:
input_file = '/home/art/data/tiny/employees_50.csv'

df = spark.read \
    .options( header = True, inferSchema = True, delimiter = ','  ) \
    .csv( input_file )

In [4]:
df.schema

StructType([StructField('EMPLOYEE_ID', IntegerType(), True), StructField('FIRST_NAME', StringType(), True), StructField('LAST_NAME', StringType(), True), StructField('EMAIL', StringType(), True), StructField('PHONE_NUMBER', StringType(), True), StructField('HIRE_DATE', StringType(), True), StructField('JOB_ID', StringType(), True), StructField('SALARY', IntegerType(), True), StructField('COMMISSION_PCT', StringType(), True), StructField('MANAGER_ID', StringType(), True), StructField('DEPARTMENT_ID', IntegerType(), True)])

In [5]:
df.printSchema()

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- DEPARTMENT_ID: integer (nullable = true)



In [6]:
df.show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |       201|           20|


In [7]:
employee = df.select( 
    'EMPLOYEE_ID', 'FIRST_NAME', 'LAST_NAME',
    'SALARY', 'DEPARTMENT_ID'
)

In [8]:
employee.show( 5 )

+-----------+----------+---------+------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|SALARY|DEPARTMENT_ID|
+-----------+----------+---------+------+-------------+
|        198|    Donald| OConnell|  2600|           50|
|        199|   Douglas|    Grant|  2600|           50|
|        200|  Jennifer|   Whalen|  4400|           10|
|        201|   Michael|Hartstein| 13000|           20|
|        202|       Pat|      Fay|  6000|           20|
+-----------+----------+---------+------+-------------+
only showing top 5 rows



In [53]:
df_employee2 = ( df
            .select( 'EMPLOYEE_ID', 'FIRST_NAME', 'LAST_NAME', 'SALARY', 'DEPARTMENT_ID', 'EMAIL' ) 
            .withColumnRenamed( 'DEPARTMENT_ID', 'DEP_ID' )
            .withColumnRenamed( 'EMAIL', 'EMAIL_OLD' )
                
              )

In [54]:
df_employee2.show( 3 )

+-----------+----------+---------+------+------+---------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|SALARY|DEP_ID|EMAIL_OLD|
+-----------+----------+---------+------+------+---------+
|        198|    Donald| OConnell|  2600|    50| DOCONNEL|
|        199|   Douglas|    Grant|  2600|    50|   DGRANT|
|        200|  Jennifer|   Whalen|  4400|    10|  JWHALEN|
+-----------+----------+---------+------+------+---------+
only showing top 3 rows



convert email to lowercase, and convert salary from usd to mxn


In [55]:
usd_to_mxn = 16.44

df_employee2 = (df_employee2
                .withColumn( 'SALARY', df_employee2.SALARY * usd_to_mxn )
                .withColumn( 'EMAIL_OLD' , F.lower( df_employee2.EMAIL_OLD  )   )
               )



In [56]:
df_employee2.show( 3 )

+-----------+----------+---------+-------+------+---------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME| SALARY|DEP_ID|EMAIL_OLD|
+-----------+----------+---------+-------+------+---------+
|        198|    Donald| OConnell|42744.0|    50| doconnel|
|        199|   Douglas|    Grant|42744.0|    50|   dgrant|
|        200|  Jennifer|   Whalen|72336.0|    10|  jwhalen|
+-----------+----------+---------+-------+------+---------+
only showing top 3 rows



In [59]:
df_employee2 = (df_employee2 
                .withColumn( 'EMAIL' , F.concat(  'EMAIL_OLD' , F.lit( '@win.com' )  )   )
                .drop( 'EMAIL_OLD' )
               )

In [60]:
df_employee2.show( 3 )

+-----------+----------+---------+-------+------+----------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME| SALARY|DEP_ID|           EMAIL|
+-----------+----------+---------+-------+------+----------------+
|        198|    Donald| OConnell|42744.0|    50|doconnel@win.com|
|        199|   Douglas|    Grant|42744.0|    50|  dgrant@win.com|
|        200|  Jennifer|   Whalen|72336.0|    10| jwhalen@win.com|
+-----------+----------+---------+-------+------+----------------+
only showing top 3 rows



### Analysis

We want to get some data about the salaries

1. Comparing employee salary 

   a) difference between his salary and the highest salary of the company,
   
   b) difference between his salary and the lowest  salary of the company,      

   c) difference between his salary and the highest salary of the department
   
   d) difference between his salary and the lowest  salary of the department
   
   
   
2. department with the highest compound salary

3. all the employees that belong to the department with a compound salary > 300

4. employees with the top salary per department




### 1a. difference between employee salary and the highest salary of the company,

and 

### 1b. difference between his salary and the lowest salary of the company, 


In [8]:
max_salary_com = ( employee.select( F.max( 'salary' ) )                 
                .withColumnRenamed( 'max(salary)', 'max_salary_com' )
                .first()                                             
                .asDict()[ 'max_salary_com' ] )
    
print( 'max_salary_com: {}'.format( max_salary_com ) )


max_salary_com: 24000


In [9]:
min_salary_com = ( employee.select( F.min( 'salary' ) )
                .withColumnRenamed( 'min(salary)', 'min_salary_com' )
                .first()                                             
                .asDict()[ 'min_salary_com' ] )
    
print( 'min_salary_com: {}'.format( min_salary_com ) )

min_salary_com: 2100


In [10]:
df_1a = employee \
        .withColumn( 'diff_max_salary',  max_salary_com - employee.SALARY  ) \
        .withColumn( 'diff_min_salary',  employee.SALARY - min_salary_com  ) 

df_1a.show()


+-----------+----------+---------+------+-------------+---------------+---------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|SALARY|DEPARTMENT_ID|diff_max_salary|diff_min_salary|
+-----------+----------+---------+------+-------------+---------------+---------------+
|        198|    Donald| OConnell|  2600|           50|          21400|            500|
|        199|   Douglas|    Grant|  2600|           50|          21400|            500|
|        200|  Jennifer|   Whalen|  4400|           10|          19600|           2300|
|        201|   Michael|Hartstein| 13000|           20|          11000|          10900|
|        202|       Pat|      Fay|  6000|           20|          18000|           3900|
|        203|     Susan|   Mavris|  6500|           40|          17500|           4400|
|        204|   Hermann|     Baer| 10000|           70|          14000|           7900|
|        205|   Shelley|  Higgins| 12008|          110|          11992|           9908|
|        206|   William|    Giet

### 1c. difference between his salary and the highest salary of the department

In [19]:
win = Window.partitionBy( 'DEPARTMENT_ID' ).orderBy( F.col( 'SALARY' ).desc() )




df = ( employee
    .withColumn( 'max_salary'    , F.max( 'SALARY' ).over( win ) )
    .withColumn( 'dif_max_salary', F.max( 'SALARY' ).over( win ) - employee.SALARY )
     )
    


df.show( )


+-----------+----------+----------+------+-------------+----------+--------------+
|EMPLOYEE_ID|FIRST_NAME| LAST_NAME|SALARY|DEPARTMENT_ID|max_salary|dif_max_salary|
+-----------+----------+----------+------+-------------+----------+--------------+
|        200|  Jennifer|    Whalen|  4400|           10|      4400|             0|
|        201|   Michael| Hartstein| 13000|           20|     13000|             0|
|        202|       Pat|       Fay|  6000|           20|     13000|          7000|
|        114|       Den|  Raphaely| 11000|           30|     11000|             0|
|        115| Alexander|      Khoo|  3100|           30|     11000|          7900|
|        116|    Shelli|     Baida|  2900|           30|     11000|          8100|
|        117|     Sigal|    Tobias|  2800|           30|     11000|          8200|
|        118|       Guy|    Himuro|  2600|           30|     11000|          8400|
|        119|     Karen|Colmenares|  2500|           30|     11000|          8500|
|   

### load

 save to parquet

In [14]:
output_file = '/home/art/data/tiny/singers.parquet'

df2.write.parquet( output_file )